# Pendahuluan

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import neurolab as nl
plt.xkcd() # biar lucu
plt.style.use('ggplot') # karena saya pengguna R juga biar bagus, Bro plotnya!
%matplotlib inline

# Single layer neural network

* collection od artificial neurons

Sebagai classifier

In [ ]:
teks = np.loadtxt('./data/data_nn_sederhana.txt')
teks 

In [ ]:
data = teks[:, :2]
data

In [ ]:
label = teks[:,2:]
label

In [ ]:
# visualisasi data
plt.scatter(data[:,0], data[:,1]);
plt.xlabel('Dimensi 1');
plt.ylabel('Dimensi 2');
plt.title('Data input');

In [ ]:
fit1_min, fit1_maks = data[:,0].min(),data[:,0].max()
fi2_min, fit2_maks = data[:,1].min(), data[:,1].max()
output = label.shape[1]

fit1 = [fit1_min, fit1_maks]
fit2 = [fit2_min, fit2_maks]

snn = nl.net.newp([fit1,fit2], output)

In [ ]:
# training 
error_progress = snn.train(data, label, epochs=100, show=20, lr=.03)

In [ ]:
plt.plot(error_progress);
plt.xlabel('Jumlah epochs');
plt.ylabel('Training error');
plt.title("Training error progress");

In [ ]:
print('Testing Data:\n')
testing_data = [[0.3, 4.2], [4.3, 0.5], [4.6, 8]]
for i in testing_data:
    print(i, '==>', snn.sim([i])[0])

# Multi-layer neural networks

Sebagai regressor

berdasarkan $y = 3x^{2} + 5$

In [ ]:
nilai_min = -20
nilai_maks = 20
jumlah_titik = 140

x = np.linspace(nilai_min, nilai_maks, jumlah_titik)
y = 3*np.square(x) + 5

# normalisasi untuk efisiensi
y /= np.linalg.norm(y)

In [ ]:
data = x.reshape(jumlah_titik,1)
label = y.reshape(jumlah_titik,1)

plt.scatter(data, label);
plt.xlabel('$x$');
plt.ylabel('$y$');

In [ ]:
# 10 neuron di hidden layer 1, 6 neuron di hidden layer 2, 1 neuron output 
mlnn = nl.net.newff([[nilai_min, nilai_maks]], [10,6,1])

# menggunakan gradien descent untuk trainingnya
mlnn.trainf = nl.train.train_gd

error_progress = mlnn.train(data, label, epochs=2000, show=100, goal=0.01)

In [ ]:
# prediksi pakai datanya sendiri
output = mlnn.sim(data)
y_pred = output.reshape(jumlah_titik)

# dense layer
x_dense = np.linspace(nilai_min, nilai_maks, jumlah_titik*2)
y_dense_pred = mlnn.sim(x_dense.reshape(x_dense.size,1)).reshape(x_dense.size)

plt.plot(x_dense, y_dense_pred, '-',label ='dense layer');
plt.plot(x, y, '.', label='nilai beneran');
plt.plot(x, y_pred, 'p', label='nilai prediksi');
plt.title('Beneran vs Prediksi');
plt.legend();

dalam project beneran harus ada pre-processing. Ga simpel kaya di sini. Berdasarkan data yg kita punya dipilih lah model nn yang cocok pada training, tweaking parameter cari yang cocok.

# Recurrent Neural Network

Buat data sikuensial. Beda sama data statis. 

* analysis of sequential data (weather, stock, traffic, video feeds).
* suitable for time-series
* require more complex architecture

Kita ga bisa sembarangan tuh pakai model2 ml sederhana atau pakai aristek nn biasa, karena kita berurusan dengan dependensi temporal dari data.

In [ ]:
# pendefinisian gelombang sinus
def dapat_data(jum_titik):
    g1 = 0.6 * np.sin(np.arange(0, jum_titik))
    g2 = 3.5 * np.sin(np.arange(0, jum_titik))
    g3 = 1.2 * np.sin(np.arange(0, jum_titik))
    g4 = 4.6 * np.sin(np.arange(0, jum_titik))
    
    a1 = np.ones(jum_titik)
    a2 = 2.2 + np.zeros(jum_titik)
    a3 = 3.1 * np.ones(jum_titik)
    a4 = 0.9 + np.zeros(jum_titik)
    
    gelombang = np.array([g1, g2, g3, g4]).reshape(jum_titik*4,1)
    amplitudo = np.array([a1, a2, a3, a4]).reshape(jum_titik*4,1)
    
    return gelombang, amplitudo

def visualisasi_output(nn, jum_titik_test):
    gelombang, amplitudo = dapat_data(jum_titik_test)
    output = nn.sim(gelombang)
    plt.plot(amplitudo.reshape(jum_titik_test*4));
    plt.plot(output.reshape(jum_titik_test*4));
    
if __name__ == '__main__':
    jum_titik = 100
    gelombang, amplitudo = dapat_data(jum_titik)

In [ ]:
# Elman RNN dengan 2 layers
nn = nl.net.newelm([[-2,2]], [10,1], [nl.trans.TanSig(), nl.trans.PureLin()])

min = -2, maks = 2 (input)

jumlah perseptron di dua layers: 10,1

dua fungsi transfer, yakni tangen hiperbolik dan linear

In [ ]:
# inisilasi fungsi untuk setiap layer
nn.layers[0].initf = nl.init.InitRand([-.1, .1], 'wb')
nn.layers[1].initf = nl.init.InitRand([-.1, .1], 'wb')

# actually initialize the net
nn.init()

In [ ]:
error_progress = nn.train(gelombang, amplitudo, epochs=2000, show=100, goal=.01)

output = nn.sim(gelombang)

plt.subplot(211);
plt.plot(error_progress);
plt.xlabel('Jumlah Epochs');
plt.ylabel('Error (MSE)');

plt.subplot(212)
plt.plot(amplitudo.reshape(jum_titik*4))
plt.plot(output.reshape(jum_titik*4))
plt.legend(['Asli', 'Prediksi'])

plt.figure();
plt.subplot(211);
visualisasi_output(nn, 150); # untuk bilangan acak di atas jumlah training
plt.xlim([0, 300]);

plt.subplot(212);
visualisasi_output(nn, 50); # untuk bilangan acak di bawah jumlah training
plt.xlim([0, 300]);

# OCR

* recognizing hand-writen character images
* dataset dari Rob Kassel dari MIT Spoken Language Systems Group yang telah dibersihkan, dinormalisasi, dan dirasterisasi oleh Ben Taskar (lektor kepala di Departemen Ilmu Komputer dan Informatika di Universitas Pennsylvania) ketika yang bersangkutan masih menjadi mahasiswa doktoral di Stanford di bawah supervisi Daphne Koller. 

In [2]:
import os
import sys # standar lib
import cv2 # opencv 
import numpy as np
file_input = "./data/letter.data"

In [12]:
# set parameters for data visualization
faktor_resize_gambar = 12
awal, akhir = 6, -1
panjang, lebar = 16, 8

# baca setiap baris data dan rescale ke 255 (rgb) hingga kita menghentikan loop pakai ctrl +c
# pake list comprehension, untuk memisahkan baris dengan tab
# reshape dari array 1 dim ke gambar berdimensi 2 kemudian mengatur skalanya dengan 
# menggunakan opencv
with open(file_input, 'r') as f:
    for baris in f.readlines():
        data = np.array([255*float(x) for x in baris.split('\t')[awal:akhir]])
        gambar = np.reshape(data, (panjang, lebar))
        gambar_terskala = cv2.resize(gambar, None,
                                     fx=faktor_resize_gambar, 
                                     fy=faktor_resize_gambar)
        print(line) # display the character
        cv2.imshow('Gambar', gambar_terskala)
        
        c = cv2.waitKey()
        if c == 27:
            break

NameError: name 'line' is not defined

In [3]:
import neurolab as nl

In [4]:
# jumlah titik data yg mau kita load
jum_data = 50
# define string containing all distinct or different chars and extract the number of class
label_ori = 'adgimno'
jum_label_ori = len(label_ori)

In [5]:
# 90 % train, 10% testing

In [6]:
jum_train = int(.9*jum_data)
jum_test = jum_data - jum_train

In [7]:
# ekstraksi parameter
awal = 6
akhir = -1

In [8]:
data = [] # fitur
labels = [] # label

with open(file_input, 'r') as f:
    for baris in f.readlines():
        nilai = baris.split('\t')
        if nilai[1] not in label_ori:
            continue
        label = np.zeros((jum_label_ori,1))
        label[label_ori.index(nilai[1])] = 1
        labels.append(label)
        
        karakter_saat_ini = np.array([float(x) for x in nilai[awal:akhir]])
        data.append(karakter_saat_ini)
        
        if len(data) >= jum_data:
            break

In [9]:
# convert labels and data to np array

In [10]:
print(type(labels), type(data))

<class 'list'> <class 'list'>


In [11]:
data = np.array(data) # float type array
labels = np.array(labels).reshape(jum_data, jum_label_ori)

In [12]:
# set number of dimension that match our need

In [13]:
jum_dim = len(data[0])

In [14]:
# feed forward nn and train it with gradien descent
nn = nl.net.newff([[0,1] for _ in range(len(data[0]))], [128,16, jum_label_ori])

# set train using gradien descent
nn.traind = nl.train.train_gd

error_progress = nn.train(data[:jum_train, :], labels[:jum_train, :], 
                          epochs = 10000, show = 1000, goal=.01)

MemoryError: Unable to allocate 2.60 GiB for an array with shape (18695, 18695) and data type float64